In [341]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [342]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
dataframe =[train_df,test_df]
survived = train_df.Survived
train_df.drop(['Survived'],axis = 1, inplace=True)

In [343]:
for dataset in dataframe:
    dataset.drop(['Cabin','Ticket'],axis = 1, inplace=True)

In [344]:
for dataset in dataframe:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

In [345]:
#train_df.head()

In [346]:
#test_df.Embarked.value_counts()

In [347]:
for dataset in dataframe:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 1, 'C': 0,'Q': 2} ).astype(int) 

In [348]:
#test_df.head()

In [349]:
for dataset in dataframe:
    dataset['Age'].fillna(dataset['Age'].median(), inplace = True)
    dataset['Age_bin'] = pd.cut(dataset['Age'], bins=[0,12,20,40,120], labels=[1,2,3,4])

In [350]:
#train_df.head()

In [351]:
for dataset in dataframe:
    dataset['Fare'].fillna(dataset['Fare'].median(), inplace = True)
    dataset['Fare_bin'] = pd.cut(dataset['Fare'], bins=[0,10.0,20.0,40.0,120.0], labels=[1,2,3,4])
    dataset.Fare_bin=  dataset.Fare_bin.fillna(1)


In [353]:
#train_df.head()
dicio={
'nan':0,'Mr.':1 ,'Mrs.':2,'Miss.':3 ,'Master.':4 ,'Planke':5,'Don.':6 ,'Rev.':7,'Ms.':8 ,'Major.':9 ,'Col.':10 ,'Capt.':11 }

for dataset in dataframe:
    dataset['NameSr']=dataset.Name.str.split().str[1].map(dicio).astype(str)#.loc[i.NameSr == 'nan','NameSr']=0
    dataset.loc[dataset.NameSr == 'nan','NameSr']=0

In [354]:
train_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Age_bin,Fare_bin,NameSr
0,1,3,"Braund, Mr. Owen Harris",0,22.0,1,0,7.2500,1,3,1,1.0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,71.2833,0,3,4,2.0
2,3,3,"Heikkinen, Miss. Laina",1,26.0,0,0,7.9250,1,3,1,3.0
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,53.1000,1,3,4,2.0
4,5,3,"Allen, Mr. William Henry",0,35.0,0,0,8.0500,1,3,1,1.0


In [355]:
dd={' Mr':1,
' Miss':2,
' Mrs':3,
' Master':4,
' Dr':5,
' Rev':6,
' Col':7,
' Major':8,
' Mlle':9,
' Lady':10,
' Sir':11,
' Mme':12,
' Don':13,
' Jonkheer':14,
' Capt':15,
' Ms':16,
' the Countess':17,}

for dataset in dataframe:
    dataset["title"] = [i.split('.')[0] for i in dataset.Name]
    dataset["title"] = [i.split(',')[1] for i in dataset.title]
    dataset.title.value_counts().keys()
    dataset.title = dataset.title.map(dd)
    
    dataset['title'] = dataset['title'].fillna(17)


In [356]:
#train_df.Fare_bin.isnull().sum()
#train_df.SibSp.value_counts()
train_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Age_bin,Fare_bin,NameSr,title
0,1,3,"Braund, Mr. Owen Harris",0,22.0,1,0,7.2500,1,3,1,1.0,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,71.2833,0,3,4,2.0,3
2,3,3,"Heikkinen, Miss. Laina",1,26.0,0,0,7.9250,1,3,1,3.0,2
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,53.1000,1,3,4,2.0,3
4,5,3,"Allen, Mr. William Henry",0,35.0,0,0,8.0500,1,3,1,1.0,1


In [357]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression


# Logistic Regression
logreg = LogisticRegression() #(C=0.1, penalty='l1', tol=1e-6)

In [358]:
Y_train=survived
X_train=train_df[['Pclass', 'Sex', 'Age_bin', 'SibSp', 'Parch', 'Fare_bin', 'Embarked','title']]
X_test = test_df[['Pclass', 'Sex', 'Age_bin', 'SibSp', 'Parch', 'Fare_bin', 'Embarked','title']]

In [359]:
X_train.head()

,Pclass,Sex,Age_bin,SibSp,Parch,Fare_bin,Embarked,title
0,3,0,3,1,0,1,1,1
1,1,1,3,1,0,4,0,3
2,3,1,3,0,0,1,1,2
3,1,1,3,1,0,4,1,3
4,3,0,3,0,0,1,1,1


In [360]:
#pd.get_dummies(X_train)


In [361]:
X_test.dtypes

Pclass         int64
Sex            int64
Age_bin     category
SibSp          int64
Parch          int64
Fare_bin    category
Embarked       int64
title        float64
dtype: object

In [362]:
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
result_train = logreg.score(X_train, Y_train)
result_val = cross_val_score(logreg,X_train, Y_train, cv=5).mean()
print('taring score = %s , while validation score = %s' %(result_train , result_val))

taring score = 0.8047138047138047 , while validation score = 0.7867968396194465


In [379]:
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
kfold = StratifiedKFold(n_splits=10)
### SVC classifier
SVMC = SVC(probability=True)
svc_param_grid = {'kernel': ['rbf'], 
                  'gamma': [ 0.001, 0.01, 0.1, 1],
                  'C': [1, 10, 50, 100,200,300, 1000]}

gsSVMC = GridSearchCV(SVMC,param_grid = svc_param_grid, cv=kfold, scoring="accuracy", n_jobs= 20, verbose = 1)

gsSVMC.fit(X_train,Y_train)

SVMC_best = gsSVMC.best_estimator_


# Best score
gsSVMC.best_score_


Fitting 10 folds for each of 28 candidates, totalling 280 fits


ImportError: [joblib] Attempting to do parallel computing without protecting your import on a system that does not support forking. To use parallel-computing in a script, you must protect your main loop using "if __name__ == '__main__'". Please see the joblib documentation on Parallel for more information

In [374]:
test_Survived_SVMC = pd.Series(SVMC_best.predict(X_test), name="SVC")


In [375]:
test_Survived_SVMC

0    0
1    1
2    0
3    0
4    1
Name: SVC, dtype: int64

In [376]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": test_Survived_SVMC
    })
submission.to_csv('titanic.csv', index=False)
print('Exported')

Exported


In [246]:
X_train=pd.get_dummies(X_train)
X_train.head()

,Pclass,Sex,SibSp,Parch,Age_bin_1,Age_bin_2,Age_bin_3,Age_bin_4,Fare_bin_1,Fare_bin_2,...,NameSr_1.0,NameSr_10.0,NameSr_11.0,NameSr_2.0,NameSr_3.0,NameSr_4.0,NameSr_6.0,NameSr_7.0,NameSr_8.0,NameSr_9.0
0,3,0,1,0,0,0,1,0,1,0,...,1,0,0,0,0,0,0,0,0,0
1,1,1,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,3,1,0,0,0,0,1,0,1,0,...,0,0,0,0,1,0,0,0,0,0
3,1,1,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,3,0,0,0,0,0,1,0,1,0,...,1,0,0,0,0,0,0,0,0,0


In [248]:
X_test=pd.get_dummies(X_test)
X_test.head()

,Pclass,Sex,SibSp,Parch,Age_bin_1,Age_bin_2,Age_bin_3,Age_bin_4,Fare_bin_1,Fare_bin_2,...,Embarked_1,Embarked_2,NameSr_0,NameSr_1.0,NameSr_10.0,NameSr_2.0,NameSr_3.0,NameSr_4.0,NameSr_7.0,NameSr_8.0
0,3,0,0,0,0,0,1,0,1,0,...,0,1,0,1,0,0,0,0,0,0
1,3,1,1,0,0,0,0,1,1,0,...,1,0,0,0,0,1,0,0,0,0
2,2,0,0,0,0,0,0,1,1,0,...,0,1,0,1,0,0,0,0,0,0
3,3,0,0,0,0,0,1,0,1,0,...,1,0,0,1,0,0,0,0,0,0
4,3,1,1,1,0,0,1,0,0,1,...,1,0,0,0,0,1,0,0,0,0


In [244]:
X_train=pd.get_dummies(X_train)
X_test=pd.get_dummies(X_test)

logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)

result_train = logreg.score(X_train, Y_train)
result_val = cross_val_score(logreg,X_train, Y_train, cv=5).mean()
print('taring score = %s , while validation score = %s' %(result_train , result_val))

ValueError: X has 23 features per sample; expecting 26

In [149]:
#y=train_df.Name.str.split()
#lista =[]
#for i in y:
#    #print(i)
#    #break
#    for j in i:
#        if len(j)<=4:
#            lista.append(j)
#    #break
#lista = set(lista)

In [150]:
#print(lista)